# DiffiT-LoRA: Checkpoint Management with Google Drive

This notebook demonstrates how to use DiffiT with automatic Google Drive checkpoint management in Google Colab.

## Features:
- ✅ Automatic Google Drive mounting and synchronization
- ✅ Checkpoint backup to Drive during training
- ✅ Automatic checkpoint loading from Drive/local with fallback
- ✅ Easy checkpoint management and download
- ✅ Works seamlessly in Colab and local environments


## 1. Setup and Installation


In [ ]:
# Install required packages
!pip install torch torchvision torchaudio
!pip install pytorch-lightning
!pip install transformers
!pip install matplotlib
!pip install tqdm

In [ ]:
!git clone https://github.com/yourusername/diffit-lora.git
    
# Change to project directory
%cd diffit-lora

# install requirements
!pip install -r requirements.txt

# Install the package
!pip install -e .


## 2. Initialize Checkpoint Manager


In [ ]:
import torch
import numpy as np
from diffit.utils.drive_integration import CheckpointManager, get_checkpoint_manager

# Initialize checkpoint manager (automatically detects Colab)
checkpoint_manager = get_checkpoint_manager(project_name="diffit-lora")

# Mount Google Drive if in Colab
if checkpoint_manager.is_colab:
    success = checkpoint_manager.mount_drive()
    if success:
        print("✅ Google Drive mounted successfully!")
        print(f"📁 Drive checkpoint path: {checkpoint_manager.drive_checkpoint_path}")
    else:
        print("⚠️ Failed to mount Google Drive, will use local storage only")
else:
    print("💻 Running locally, using local checkpoint storage")

print(f"📂 Local checkpoint path: {checkpoint_manager.local_checkpoint_dir}")


## 3. Save Checkpoint Example


In [ ]:
# Example: Save a checkpoint with automatic Drive backup
checkpoint_data = {
    'state_dict': {},  # Your model's state_dict would go here
    'epoch': 10,
    'metrics': {'loss': 0.5, 'accuracy': 0.85},
    'model_config': {'d_model': 128, 'num_heads': 2}
}

# Save checkpoint (automatically backs up to Drive if available)
saved_path = checkpoint_manager.save_checkpoint(
    checkpoint_data, 
    filename="example_checkpoint",
    backup_to_drive=True
)

print(f"💾 Checkpoint saved to: {saved_path}")
print("✅ Automatic Drive backup completed!")


## 4. Load Checkpoint Example


In [ ]:
# Load checkpoint (automatically tries Drive first, then local)
checkpoint_name = "example_checkpoint"

print(f"📥 Loading checkpoint: {checkpoint_name}")
loaded_checkpoint = checkpoint_manager.load_checkpoint(checkpoint_name)

if loaded_checkpoint is not None:
    print("✅ Checkpoint loaded successfully!")
    print(f"   Epoch: {loaded_checkpoint.get('epoch', 'Unknown')}")
    print(f"   Metrics: {loaded_checkpoint.get('metrics', {})}")
    print(f"   Model config: {loaded_checkpoint.get('model_config', {})}")
else:
    print("❌ Failed to load checkpoint")
    
# List all available checkpoints
print("\n📋 Available checkpoints:")
checkpoints = checkpoint_manager.list_checkpoints()
if checkpoints['local']:
    print(f"  Local: {', '.join(checkpoints['local'])}")
if checkpoints['drive']:
    print(f"  Drive: {', '.join(checkpoints['drive'])}")
